# Elasticsearch
엘라스틱서치의 기본 원리와 사용법에 대해서 배웁니다.

[thejungwon/search-engine-tutorial](https://github.com/thejungwon/search-engine-tutorial)의 튜토리얼 일부분을 수정한 코드입니다.
____

## 1. Elasticsearch의 Python 라이브러리 설치

- Elasticsearch를 미리 설치한 후에 진행해주세요 (노션 설치 설명 참고)

In [1]:
!pip install elasticsearch

     |████████████████████████████████| 378 kB 21.4 MB/s eta 0:00:01


In [1]:
import tqdm
import json
from pprint import pprint
from elasticsearch import Elasticsearch

In [2]:
!curl -XGET localhost:9200

{
  "name" : "f867c65de115",
  "cluster_name" : "elasticsearch",
  "cluster_uuid" : "586snTJiQIef5XYkmM6jhw",
  "version" : {
    "number" : "7.17.3",
    "build_flavor" : "default",
    "build_type" : "deb",
    "build_hash" : "5ad023604c8d7416c9eb6c0eadb62b14e766caff",
    "build_date" : "2022-04-19T08:11:19.070913226Z",
    "build_snapshot" : false,
    "lucene_version" : "8.11.1",
    "minimum_wire_compatibility_version" : "6.8.0",
    "minimum_index_compatibility_version" : "6.0.0-beta1"
  },
  "tagline" : "You Know, for Search"
}


## 2. Index 세팅
- 한글 데이터를 다루기 때문에, 노리 형태소 분석기를 탑재합니다.

In [3]:
import pprint  
INDEX_NAME = "wiki-sample"

INDEX_SETTINGS = {
    "settings": {
        "analysis": {
            "filter": {
                "my_shingle": {
                    "type": "shingle"
                }
            },
            "analyzer": {
                "my_analyzer": {
                    "type": "custom",
                    "tokenizer": "nori_tokenizer",
                    "decompound_mode": "mixed",
                    "filter": ["my_shingle"]
                }
            },
            "similairty": {
                "my_similarity": {
                    "type": "BM25"
                }
            }
        }
    },

    "mappings": {
        "properties": {
            "document_text": {
                "type": "text",
                "analyzer": "my_analyzer"
            }
        }
    }
}

## 3. 문서 준비

In [6]:
# 위키피디아 데이터 로드

dataset_path = "../../data/wikipedia_documents.json"
with open(dataset_path, "r") as f:
    wiki = json.load(f)
    
print(len(wiki))
print(type(wiki))

60613
<class 'dict'>


In [7]:
wiki['0']

{'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.',
 'corpus_source': '위키피디아',
 'url': 'TODO',
 'domain': None,
 'title': '나라 목록',
 'author': None,
 'html': None,
 'document_id': 0}

In [17]:
# # 일부분만 삽입해보기

# small_wiki = {'0': wiki['0'],
#               '1': wiki['1'],
#               '2': wiki['2'],
#               '3': wiki['3'],
#               '4': wiki['4'],
#               '5': wiki['5'],
#               '6': wiki['6'],
#               '7': wiki['7'],
#               '8': wiki['8'],
#               '9': wiki['9'],
#              }
# print(len(small_wiki))
# print(type(small_wiki))
# pprint(small_wiki['0'])

In [9]:
import re

def preprocess(text):
    text = re.sub(r"\n", " ", text)
    text = re.sub(r"\\n", " ", text)
    text = re.sub(r"#", " ", text)
    text = re.sub(r"[^A-Za-z0-9가-힣.?!,()~‘’“”"":%&《》〈〉''㈜·\-\'+\s一-龥サマーン]", "", text)  
    text = re.sub(r"\s+", " ", text).strip()  # 두 개 이상의 연속된 공백을 하나로 치환
    
    return text

In [10]:
wiki_contexts = list(dict.fromkeys([v["text"] for v in wiki.values()]))  # text 부분만
wiki_contexts = [preprocess(text) for text in wiki_contexts]
wiki_articles = [
    {"document_text": wiki_contexts[i]} for i in range(len(wiki_contexts))
]

In [11]:
wiki_contexts[0]

'이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다. 이 목록은 명료화를 위해 두 부분으로 나뉘어 있다. 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다. 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다. 두 목록은 모두 가나다 순이다. 일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 포함 기준 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 국가 문서에서 설명하고 있다.'

## 4. Elastichsearch 접속

In [12]:
try:
    es.transport.close()
except:
    pass

# config = {'host':'localhost', 'port':9200}
# es = Elasticsearch([config])

es = Elasticsearch()

print('Elastic search ping:', es.ping())
print('Elastic search info:')
es.info()

Elastic search ping: True
Elastic search info:


/opt/conda/lib/python3.8/site-packages/elasticsearch/connection/base.py:208: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


{'name': 'f867c65de115',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': '586snTJiQIef5XYkmM6jhw',
 'version': {'number': '7.17.3',
  'build_flavor': 'default',
  'build_type': 'deb',
  'build_hash': '5ad023604c8d7416c9eb6c0eadb62b14e766caff',
  'build_date': '2022-04-19T08:11:19.070913226Z',
  'build_snapshot': False,
  'lucene_version': '8.11.1',
  'minimum_wire_compatibility_version': '6.8.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

## 5. 인덱스 생성

In [14]:
if es.indices.exists(INDEX_NAME):
    es.indices.delete(index=INDEX_NAME)
es.indices.create(index=INDEX_NAME, body=INDEX_SETTINGS)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'wiki-sample'}

In [15]:
es.indices.exists("wiki-sample")

True

## 6. 문서삽입

In [17]:
# 5-10분 정도 소요됩니다.

for i, text in enumerate(tqdm(wiki_articles)):
    try:
        es.index(index=INDEX_NAME, id=i, body=text)
    except:
        print(f"Unable to load document {i}.")

TypeError: 'module' object is not callable

## 7. 삽입된 문서 확인

In [16]:
doc = es.get(index=INDEX_NAME, id=0)
pprint(doc)

NotFoundError: NotFoundError(404, '{"_index":"wiki-sample","_type":"_doc","_id":"0","found":false}')

## 8. Term vector 확인

In [27]:
# 출력문이 기니까 pprint() 주석 풀고 한 번 확인해보세용

tv = es.termvectors(index=INDEX_NAME, id=1, body={"fields" : ["document_text"]})
# pprint(tv)

## 9. 검색

In [1]:
query = "주기표는 무엇인가?"
query_res = es.indices.analyze(index=INDEX_NAME,
                                 body={
                                       "analyzer" : "my_analyzer",
                                        "text" : query
                                 }
                        )
pprint(query_res)

NameError: name 'es' is not defined

In [34]:
# topk=5로 retrieval

query = "주기표는 무엇인가?"
body = {
        "query": {
            "bool": {
                "must": [{"match": {"document_text": query}}],
            }
        }
    }

res = es.search(index=INDEX_NAME, body=body, size=5)
pprint(res)

{'_shards': {'failed': 0, 'skipped': 0, 'successful': 1, 'total': 1},
 'hits': {'hits': [{'_id': '9',
                    '_index': 'wiki-sample',
                    '_score': 8.72252,
                    '_source': {'document_text': '주기율표(週期律表, 주기률표, periodic '
                                                 'table) 또는 주기표(週期表)는 원소를 구분하기 '
                                                 '쉽게 성질에 따라 배열한 표로, 러시아의 드미트리 '
                                                 '멘델레예프가 처음 제안했다. 1913년 헨리 '
                                                 '모즐리는 멘델레예프의 주기율표를 개량시켜서 '
                                                 '원자번호순으로 배열했는데, 이는 현대의 원소 '
                                                 '주기율표와 유사하다. 가장 많이 쓰이는 주기율표에는 '
                                                 '단주기형과 장주기형이 있다. 단주기형 주기율표는 '
                                                 '1주기와 3주기를 기준으로 하고, 4주기 아래로는 '
                                                 '전형원소와 전이원소가 같은 칸에 있다. 이 단주기형 '
                        

In [35]:
# top5 문서 id, score 출력
for hit in res['hits']['hits']:
    print("Doc ID: %3r  Score: %5.2f" % (hit['_id'], hit['_score']))


Doc ID: '9'  Score:  8.72
Doc ID: '2'  Score:  1.72
Doc ID: '1'  Score:  1.69
Doc ID: '5'  Score:  0.20
Doc ID: '0'  Score:  0.20
